In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
from tensorflow.keras import layers
import string

2025-11-08 08:07:07.717897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762589227.894149      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762589227.944270      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
BATCH_SIZE = 128
NUM_HEADS = 12
NUM_BLOCKS = 8
EMBED_DIM = 384
DENSE_DIM = 1536
DROPOUT_RATE = 0.1
CHUNK_LENGTH = 150

In [3]:
df = pd.read_csv('/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv')

In [4]:
text = '<sos>' + df.values[0][0] + '<eos>'
text = re.sub(r'\s+', ' ', str(text)).strip()

In [5]:
import re

words = text.split()

# Count unique words
unique_words = set(words)
print(f"Total words: {len(words)}")
print(f"Unique words: {len(unique_words)}")


Total words: 182499
Unique words: 23843


In [6]:
def chunk_text_by_words(text, max_words, stride=None):
    words = text.split()
    if stride is None:
        stride = max_words // 2
    chunks = []
    for i in range(0, len(words) - max_words, stride):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [7]:
chunks = chunk_text_by_words(text, CHUNK_LENGTH+1)

In [8]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 23850
sequence_length = CHUNK_LENGTH+1

vectorizer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization
)

vectorizer.adapt(chunks)

I0000 00:00:1762589240.035727      39 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [9]:
def make_dataset(chunks):
    tokens = vectorizer(chunks)
    tokens_inp = tokens[:,:CHUNK_LENGTH]
    tokens_out = tokens[:,1:]
    ds = tf.data.Dataset.from_tensor_slices((tokens_inp,tokens_out))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.shuffle(1024).prefetch(16).cache()
    return ds

In [10]:
ds = make_dataset(chunks)

In [11]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim):
        super().__init__()
        self.positional_embedding = tf.keras.layers.Embedding(input_dim = sequence_length, output_dim = output_dim, mask_zero=False)
        self.token_embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= output_dim, mask_zero=True)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.positional_embedding(positions)
        return embedded_tokens + embedded_positions

In [12]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, dense_dim, dropout_rate):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                           key_dim=embed_dim//num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense_proj = tf.keras.models.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='gelu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inputs):
        attn_out = self.attention(query=inputs,
                            key=inputs,
                            value=inputs,
                            use_causal_mask=True)
        norm1_out = self.layernorm1(attn_out+inputs)
        drop1_out = self.dropout1(norm1_out)
        dense_proj_out = self.dense_proj(drop1_out)
        norm2_out = self.layernorm2(drop1_out+dense_proj_out)
        drop2_out = self.dropout2(norm2_out)
        return drop2_out

In [13]:
inputs = tf.keras.layers.Input(shape=(None,))
embeddings = PositionalEmbedding(sequence_length, vocab_size, EMBED_DIM)(inputs)
x = embeddings
for layer in range(NUM_BLOCKS):
    x = TransformerDecoder(NUM_HEADS, EMBED_DIM, DENSE_DIM, DROPOUT_RATE)(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(vocab_size,activation='softmax')(x)
transformer = tf.keras.models.Model(inputs, output)

In [14]:
transformer.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 384)      │     9,216,384 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_2           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_3           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_4           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_5           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_6           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_7           │ (None, None, 384)      │     1,774,464 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, None, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, None, 23850)    │     9,182,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,594,346 (124.34 MB)

 Trainable params: 32,594,346 (124.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=3e-4)

transformer.compile(loss = loss_fn,
                    metrics = ['accuracy'],
                    optimizer=opt)
transformer.fit(ds, epochs = 50)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
I0000 00:00:1762589274.415279      96 service.cc:148] XLA service 0x79e030002d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762589274.416022      96 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1762589276.110804      96 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1762589277.459105      96 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1762589298.255718      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most o

19/19 ━━━━━━━━━━━━━━━━━━━━ 64s 509ms/step - accuracy: 0.0204 - loss: 9.5103
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 509ms/step - accuracy: 0.0323 - loss: 7.4166
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 509ms/step - accuracy: 0.0315 - loss: 6.8292
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.0287 - loss: 6.7930
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 509ms/step - accuracy: 0.0308 - loss: 6.7803
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 510ms/step - accuracy: 0.0299 - loss: 6.7762
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 510ms/step - accuracy: 0.0303 - loss: 6.7742
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.0304 - loss: 6.7717
Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.0300 - loss: 6.7713
Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.0302 - loss: 6.7700
Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.0304 - loss: 6.7692
Epoch 12/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/ste